In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_recall_curve, auc, precision_score, recall_score, f1_score
)
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, make_scorer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import cross_validate

In [ ]:
# Load dataset
df = pd.read_csv("C:/Users/maxhi/Documents/GitHub/Credit_Card_Fraud_Detection/creditcard.csv")

In [ ]:
# Basic data check
print(df.shape)
print(df['Class'].value_counts(normalize=True))  # Shows class imbalance - fraud rate of ~0.17%
print(df.isnull().sum())